In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
import selenium
from selenium import webdriver
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here
#DRIVER_PATH = r'C:\Users\iDAFAdmin\Downloads\chromedriver_win32\chromedriver.exe'
wd = webdriver.Chrome(executable_path= r'C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Documents\Clock by Dreck\chromedriver_win32')

WebDriverException: Message: 'chromedriver_win32' executable may have wrong permissions. Please see https://sites.google.com/a/chromium.org/chromedriver/home


In [7]:
wd.get('https://google.com')

In [8]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

In [9]:
wd.quit()

In [4]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [8]:
pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import PIL
from PIL import Image
import hashlib
import os
import time
import io

In [5]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [19]:
def search_and_download(search_term:str,driver_path:str,target_path=r'C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images',number_images=90):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)
        return('Your Images are ready')

In [22]:
search_term = 'Black American smiling faces'

search_and_download(search_term = search_term,
                   driver_path = DRIVER_PATH
                   )


Found: 100 search results. Extracting links from 0:100
Found: 90 image links, done!
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQzuWpGxgXIMU0-4Y0k0X3XcOnX5yk2sOoVFA&usqp=CAU - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\d1ee65af09.jpg
SUCCESS - saved https://www.nicepng.com/png/detail/164-1649946_happy-smiling-emoticon-face-vector-smile.png - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\984eb96d3d.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTk7lo4dgvLpWTp_uO68IQSsFVTqS1eoYVYHA&usqp=CAU - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\8ea6a68d45.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREFRC2MF_Pp86xqH_HsCRVRL1qMV8PcMAI2A&usqp=CAU - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\108

SUCCESS - saved https://ak.picdn.net/shutterstock/videos/8541835/thumb/11.jpg - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\6e5c96c83c.jpg
SUCCESS - saved https://media.istockphoto.com/photos/happy-african-family-looking-at-camera-on-bed-top-view-picture-id1158481708 - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\8a9389a57b.jpg
SUCCESS - saved https://upload.wikimedia.org/wikipedia/commons/thumb/8/84/A_smiley_face_balloon_from_a_Gregory_FUNNY-B%27LOONS_ad_on_page_20_of_The_Billboard_March_18_1922.jpg/440px-A_smiley_face_balloon_from_a_Gregory_FUNNY-B%27LOONS_ad_on_page_20_of_The_Billboard_March_18_1922.jpg - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\17732603da.jpg
SUCCESS - saved https://image.shutterstock.com/image-photo/close-portrait-smiling-young-african-260nw-383193250.jpg - as C:\Users\iDAFAdmin\OneDrive - Data

SUCCESS - saved https://image1.masterfile.com/getImage/NzAwLTAzODQ4ODgyZW4uMDAwMDAwMDA=AB6sdi/700-03848882en_Masterfile.jpg - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\44aa0277a8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTphFNLs_jhT0BquUCrhQ4ATt-oVUPVMVbbWA&usqp=CAU - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\3a8f0bc5a3.jpg
SUCCESS - saved https://us.123rf.com/450wm/kurtvate/kurtvate1603/kurtvate160300029/54026072-the-face-of-a-beautiful-young-african-american-woman-in-closeup-smiling-with-curly-black-hair.jpg?ver=6 - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smiling_faces\3e915878a8.jpg
SUCCESS - saved https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTW4iSbnx3DfMGqVoqYjvy4JEL_2t3fmab7Zg&usqp=CAU - as C:\Users\iDAFAdmin\OneDrive - Data Wrangling\Desktop\Scraper./images\black_american_smili

In [ ]:
def filter_students(class_names):
	new_class_names = []
	for c in class_names:
		if c.startswith("E"):
			new_class_names.append(c)

In [1]:
#Random Quotes generator
import requests

url = 'https://api.quotable.io/random'

r = requests.get(url)
quote = r.json()
#print(quote)
print(quote['content'])
print('     -',quote['author'])

Every great advance in science has issued from a new audacity of the imagination.
     - John Dewey
